In [0]:
import sys
import os
import pandas as pd
import numpy as np
import scipy as sp
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
import Bio
from Bio.Blast.Applications import NcbiblastxCommandline

In [0]:
bombyx = "/gpfs_fs/home/eckertlab/genomes/Bombyx_mori/protein/protein.fa"
plutella = "/gpfs_fs/home/eckertlab/genomes/Plutella_xylostella/protein/protein.fa"

## Create blast databases

```bash
~/g/src/ncbi-blast-2.2.30+/bin/makeblastdb -in protein.fa -dbtype prot
```

In [0]:
blastx = "/home/cfriedline/g/src/ncbi-blast-2.2.30+/bin/blastx"
assembly = "/home/cfriedline/eckertlab/projects/gypsy_moth/assemblies/masurca3/CA/10-gapclose/genome.ctg.fasta"
analysis_dir = "/home/cfriedline/eckertlab/gypsy_indiv/raw_demult/analysis/samtools1.3_masurca3/"

In [0]:
outfiles = []
clines = []
for org, db in list({'bombyx': bombyx, 'plutella': plutella}.items()):
    outdir = os.path.join(analysis_dir, "blast")
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    out = os.path.join(outdir, "%s.xml" % org)
    outfiles.append(out)
    cline = NcbiblastxCommandline(cmd=blastx,
                                 out=out,
                                 db=db,
                                 query=assembly,
                                 outfmt=5,
                                 max_target_seqs=5,
                                 num_threads=10,
                                 evalue=1e-5)
    clines.append(cline)

In [0]:
with open(os.path.join(analysis_dir, "blast.sh"), "w") as o:
    for cline in clines:
        o.write("%s\n" % cline)

```bash
~/bin/parallel --no-notice --bar -a blast.sh
```